## loggingにまつわる疑問

注意: 特に指定がない限り、セルの実行前にPythonランタイムをRestartすること

In [2]:
# logging.basicConfig() は、すでに生成済みのloggerインスタンスにも影響するか？
# → 影響する

from logging import basicConfig, getLogger, DEBUG

def play_basic_config():
    logger = getLogger(__name__)
    logger.info('information 1')
    basicConfig(level=DEBUG)
    logger.info('information 2')
    
play_basic_config()

INFO:__main__:information 2


In [1]:
# logging.basicConfig()は冪等か？つまり、複数呼び出してもStreamHandlerを1つだけ追加するか？
# → 冪等 (force=Trueの場合を除く)

from logging import basicConfig, getLogger

def play_basic_config_idempotency():
    root = getLogger('root')
    print(f"Default: {root.handlers=}")
    basicConfig()
    print(f"basicConfig called (1): {root.handlers=}")
    basicConfig()
    print(f"basicConfig called (2): {root.handlers=}")
    basicConfig()
    print(f"basicConfig called (3), but different parameter: {root.handlers=}")

play_basic_config_idempotency()

Default: root.handlers=[]
basicConfig called (1): root.handlers=[<StreamHandler stderr (NOTSET)>]
basicConfig called (2): root.handlers=[<StreamHandler stderr (NOTSET)>]
basicConfig called (3), but different parameter: root.handlers=[<StreamHandler stderr (NOTSET)>]


In [1]:
# basicConfig()を呼び出さずにロギングしたら、ハンドラが生成されるの？
# → されない。logging.lastResort ハンドラが利用される

from logging import getLogger

def when_handler_generated():
    root = getLogger('root')
    print(f"handlers before logging: {root.handlers}")
    root.warning('warning message')
    print(f"handlers after logging: {root.handlers}")

when_handler_generated()

warning message


handlers before logging: []
handlers after logging: []


In [4]:
# propagateがFalseの場合も、子のloggerはlogging.lastResortを利用できるの？
# → できる

from logging import getLogger

def child_logger_default_handler_without_propagate():
    child = getLogger('child')
    child.propagate = False
    print(f"{child.handlers}")
    child.warning('warning message')

child_logger_default_handler_without_propagate()

warning message


[]


In [3]:
# 子のloggerが親のloggerのログレベルを継承するというのは、どのタイミングの話？
# → ログレベルの継承は、実際に数字を継承する訳ではなく、ログレベルが0の場合に親のログレベルを参照するようだ

from logging import basicConfig, getLogger, DEBUG, INFO

def when_level_inherited():
    # 子のloggerの生成前に、ルートロガーのlevelをINFOに設定する
    root = getLogger('root')
    basicConfig(level=INFO)

    # 子のloggerを生成する
    child = getLogger('child')
    print(f"Level just after generation: {root.level=}, {child.level=}")

    root.info('root information')
    child.info('child information')
    print(f"Level after logging: {root.level=}, {child.level=}")

    # ルートロガーのlevelをDEBUGに変更する
    basicConfig(force=True, level=DEBUG)
    print(f"Level after change root level: {root.level=}, {child.level=}")

    root.debug('root debug')
    child.debug('child debug')
    print(f"Level after logging: {root.level=}, {child.level=}")

when_level_inherited()

INFO:root:root information
INFO:child:child information
DEBUG:root:root debug
DEBUG:child:child debug


Level just after generation: root.level=20, child.level=0
Level after logging: root.level=20, child.level=0
Level after change root level: root.level=10, child.level=0
Level after logging: root.level=10, child.level=0


In [3]:
# Jupyter Notebookのセルで`__name__`は何？
# → __main__

print(f"{__name__=}")

__name__='__main__'


In [4]:
# ランタイムがJupyter Notebookであることを確認する楽な方法は？
# → get_ipython が定義されているかを確認すれば良い。

print(f"{'get_ipython' in globals()=}")

# get_ipython() を実行する方法。エラーハンドリングが必要になる。
print(f"This is a Jupyter environment, {get_ipython()}") # type: ignore

# __IPYTHON__ を参照する方法。`__IPYTHON__`は扱いが特殊らしく、globals()に定義されていない。
print(f"{__IPYTHON__=}") # type: ignore
print(f"{'__IPYTHON__' in globals()=}")

# ipykernelがモジュールに存在するかを確かめる方法
import sys

print(f"{'ipykernel' in sys.modules=}")

'get_ipython' in globals()=True
This is a Jupyter environment, <ipykernel.zmqshell.ZMQInteractiveShell object at 0x10408c470>
__IPYTHON__=True
'__IPYTHON__' in globals()=False
'ipykernel' in sys.modules=True


## Jupyter Notebookでloggingを用いる上での考察

In [2]:
from logging import basicConfig, root, DEBUG, WARNING

def some_algorithm():
    # Jupyter Notebookでloggingをしていると、同じセルを複数回実行した際にStreamHandlerが複数追加されることが悩ましい。
    # basicConfigはStreamHandlerの追加が冪等なため、ルートロガーの利用またはpropagateを認めることになるが、メリットの方が大きいと判断した。
    basicConfig(level=DEBUG if 'get_ipython' in globals() else WARNING)

    # どうせルートロガーを使うなら明示的に使ったほうがトラブルが少ないと考える。
    root.debug('hello')

some_algorithm()

DEBUG:root:hello
